In [1]:
%load_ext autoreload
%autoreload 2

from scholarly import scholarly
from tqdm import tqdm
from glob import glob
from parse_bib import *
list_of_bibs=[]
import arxiv
import json



In [2]:
### Loading old dataset

file1 = open("hateSpeechPapers.txt","r") 
list1=file1.readlines()
file1.close() 

In [3]:
papers_dict={}
key=None
for file in list1:
    file=file.strip('\n')
    if(file[-1:]==":"):
        key=file[:-1]
    else:
        name=file
        if(name != ""):
            try:
                papers_dict[key].append(name)
            except KeyError:
                papers_dict[key]=[name]
            

In [4]:
import re



In [6]:
### set search_pub to True if u have keywords 
### if u have titles, set it to false
search_pub=True

In [14]:
if search_pub==True:
    search_name='Using Knowledge Graphs to improve Hate Speech Detection'
    year_low = 2016
    year_high =2022
    temp=scholarly.search_pubs(search_name,year_low=year_low,year_high=year_high)
    count=0

    while(count<10):
        try:
            count=count+1
            print(count)
            temp_pub=next(temp,None)
            if(temp_pub is None):
                break
            list_of_bibs.append(temp_pub['bib'])
        except ConnectionRefusedError:
            pass
        filename = 'own_papers' +'_'+str(year_low)+'_'+str(year_high)+'.bib'
        output_file = open('bib_folder/'+filename, 'w', encoding='utf-8')
        for text in list_of_bibs:
            output_file.write(text)
        output_file.close()    
else:
    count=0
    for key in papers_dict:
        if(count!=4):
            count+=1
            continue
            
        list_of_bibs=[]
        for paper in tqdm.tqdm(papers_dict[key][10:20]):
            search_name=paper
            temp=scholarly.search_pubs(search_name)
            temp_pub=next(temp,None)
            if(temp_pub is None):
                    print(paper)
                    pass
            else:
                list_of_bibs.append(temp_pub.bib)
            if(len(list_of_bibs)!=0):
                filename = key+'_from_old_repo.bib'
                output_file = open('bib_folder/'+filename, 'w', encoding='utf-8')
                for text in list_of_bibs:
                    output_file.write(text)
                output_file.close()

1


TypeError: write() argument must be str, not dict

{'container_type': 'Publication',
 'source': <PublicationSource.PUBLICATION_SEARCH_SNIPPET: 1>,
 'bib': {'title': 'Hate me, hate me not: Hate speech detection on facebook',
  'author': ["F Dell'Orletta"],
  'pub_year': '2017',
  'venue': 'Proceedings of the First …',
  'abstract': 'This improvement is particularly significant for the classification of the Hate class, with F-score of about 72%. These results pave the way to the employment of our system in a real-use context  Filtering offensive language in online communities using grammatical relations'},
 'filled': False,
 'gsrank': 1,
 'pub_url': 'http://ceur-ws.org/Vol-1816/paper-09.pdf',
 'author_id': ['', 'rJ1LGyYAAAAJ', 'uhInFTQAAAAJ'],
 'num_citations': 161,
 'url_scholarbib': '/scholar?q=info:i5zC6Yp5Hd4J:scholar.google.com/&output=cite&scirp=0&hl=en',
 'url_add_sclib': '/citations?hl=en&xsrf=&continue=/scholar%3Fq%3DUsing%2BKnowledge%2BGraphs%2Bto%2Bimprove%2BHate%2BSpeech%2BDetection%26hl%3Den%26as_sdt%3D0,33%26as_ylo%3D2016%2

In [16]:
str(list_of_bibs)

'[{\'title\': \'Hate me, hate me not: Hate speech detection on facebook\', \'author\': ["F Dell\'Orletta"], \'pub_year\': \'2017\', \'venue\': \'Proceedings of the First …\', \'abstract\': \'This improvement is particularly significant for the classification of the Hate class, with F-score of about 72%. These results pave the way to the employment of our system in a real-use context  Filtering offensive language in online communities using grammatical relations\'}]'

In [3]:
import json 
from os import path


In [4]:
if path.exists("author.json"):
    with open("author.json", "r") as infile: 
        author_dict=json.load(infile)
    


In [5]:
language_tags= ['english','arabic','italian','portugese','multilingual','german','french','indonesian','polish','kenyan','amharic','dutch']
method_tags =['detection','generation','bias','counter','countering','survey','classification','classify','crime','dataset']
type_of_hate_tags = ['racial','ethnic','offensive']


reset_publications=False


def get_author_link(author_name):
    try:
        author_web=author_dict[author_name]
    except:
        author_tuple=author_name.split(" ")
        author_plus_name=author_tuple[0]
        for name in author_tuple[1:]:
            author_plus_name=author_plus_name+'+'+name
        author_web='https://scholar.google.com/citations?view_op=search_authors&hl=en&mauthors='+author_plus_name+'&btnG='
        author_dict[author_name]=author_web
    return author_web
#the_file.write('url_pdf : "/publication/'+entry['ID']+'/manuscript.pdf"\n'


total_tags = language_tags+method_tags+type_of_hate_tags

def bibtomark(inputfile,tags=None):
    print(inputfile)
    try:
        with open(inputfile, encoding="utf8") as bibtex_file:
            bibtex_str = bibtex_file.read()
#             print(bibtex_str)
    except EnvironmentError:  # parent of IOError, OSError *and* WindowsError where available
        print('File '+inputfile+' not found or some other error...')
    
    # It takes the type of the bibtex entry and maps to a corresponding category of the academic theme
    # Publication type.
    # Legend:
    # 0 = Uncategorized
    # 1 = Conference paper
    # 2 = Journal article
    # 3 = Preprint / Working Paper
    # 4 = Report
    # 5 = Book
    # 6 = Book section
    # 7 = Thesis
    # 8 = Patent
    pubtype_dict = {
        'phdthesis': '"7"',
        'mastersthesis': '"7"',
        'Uncategorized': '"0"',
        'inproceedings': '"1"',
        'conference': '"1"',
        'article': '"2"',
        'submitted': '"3"',
        'techreport': '"4"',
        'book': '"5"',
        'incollection': '"6"',
    }
    bib_database = bibtexparser.loads(bibtex_str)
    #print(bib_database.entries)
    for entry in bib_database.entries:
        #print(entry)
        filedir = 'content/publication/'+entry['ID'] 
        if not os.path.exists(filedir):
            os.mkdir(filedir)
        filenm = 'content/publication/'+entry['ID']+'/index.md'
        
        #If the same publication exists, then skip the creation. I customize the .md files later, so I don't want them overwritten. Only new publications are created.
        if os.path.isfile(filenm) and reset_publications==False:
            continue
        
        
        with open(filenm, 'w', encoding='utf8') as the_file:
            the_file.write('---\n')
            the_file.write('title: "'+supetrim(entry['title'])+'"\n')
            print('Parsing ' + entry['ID'])
            
            temp_list=arxiv.query(query=supetrim(entry['title']), max_results=1)
            
            if(len(temp_list)!=0 and temp_list[0]['title']!=supetrim(entry['title'])):
                print('arxiv:',temp_list[0]['title'])
                print('gscholar:',supetrim(entry['title']))
                print('++++++++++++++')
                temp_list=[]
            
            if 'year' in entry:
                date = entry['year']
                if 'month' in entry:
                    if RepresentsInt(entry['month']):
                        month = entry['month']
                    else:
                        month = str(month_string_to_number(entry['month']))
                    date = date+'-'+ month.zfill(2)
                else:
                    date = date+'-01'
                the_file.write('date: "'+date+'-01"\n')
                
            # Treating the authors
            if 'author' in entry:
                authors = entry['author'].split(' and ')
                #print(authors)
                the_file.write('authors: [')
                authors_str = ''
                for author in authors:
                    author_strip = supetrim(author)
                    #print(author_strip)
                    author_split = author_strip.split(',')
                    if len(author_split)==2:
                        author_strip = author_split[1].strip() + ' ' +author_split[0].strip()
#                     author_split = author_strip.split(' ')
#                     author_strip = author_split[0][0]+'. '+' '.join(map(str, author_split[1:]))
                    if author_strip == 'll':
                        author_strip = 'admin'
                    author_web = get_author_link(author_strip)
                    if author_web:
                        #authors_str = author
                        authors_str+='"['+author_strip+'](' + author_web + ')",'
                    else:
                        authors_str = authors_str+ '"'+author_strip+'",'
            the_file.write(authors_str[:-1]+']\n')
            
            # Treating the keywords
            title_tokens=supetrim(entry['title']).lower().split(" ")
            
            keywords=[]
            if(tags!=None):
                for temp in tags:
                    keywords.append(temp)
            
            
            flag_language = False
            for tag in total_tags:
                if tag in title_tokens:
                    if(tag in language_tags):
                        flag_language=True
                    elif(tag == 'counter' or tag == 'countering' ):
                        keywords.append('Counter speech')
                    elif(tag == 'crime'):
                        keywords.append('Hate crime')
                        
                    elif(tag in ['classification','classify','detection']):
                        keywords.append('Detection')
                    else:
                        keywords.append(tag.capitalize())
            
            if(flag_language==False):
                keywords.append('English')
            #print(keywords)    
                
            if len(keywords) >0:
                keywords=list(set(keywords))
                the_file.write('tags : [')
                keyword_str = ''
                for keyword in keywords:
                    keyword_str = keyword_str+ '"'+keyword+'",'
                #print(keyword_str)
                the_file.write(keyword_str[:-1]+']\n')
            else:
                the_file.write('tags : []\n')
            
            # Treating the publication type
            if 'ENTRYTYPE' in entry:
                if 'booktitle' in entry and ('Seminar' in supetrim(entry['booktitle'])):
                    the_file.write('publication_types : ['+pubtype_dict['conference']+']\n')
                elif 'booktitle' in entry and ('Workshop' in supetrim(entry['booktitle'])):
                    the_file.write('publication_types : ['+pubtype_dict['conference']+']\n')
                elif 'note' in entry and ('review' in supetrim(entry['note'])):
                    the_file.write('publication_types : ['+pubtype_dict['submitted']+']\n')
                elif 'note' in entry and ('Conditional' in supetrim(entry['note'])):
                    the_file.write('publication_types : ['+pubtype_dict['submitted']+']\n')
                else:
                    the_file.write('publication_types : ['+pubtype_dict[entry['ENTRYTYPE']]+']\n')
            
            # Treating the publication journal, conference, etc.
            if 'booktitle' in entry:
                the_file.write('publication : "_'+supetrim(entry['booktitle'])+'_"\n')
            elif 'journal' in entry:
                the_file.write('publication : "_'+supetrim(entry['journal'])+'_"\n')
            elif 'school' in entry:
                the_file.write('publication : "_'+supetrim(entry['school'])+'_"\n')
            elif 'institution' in entry:
                the_file.write('publication : "_'+supetrim(entry['institution'])+'_"\n')
                
            # I never put the short version. In the future I will use a dictionary like the authors to set the acronyms of important conferences and journals
            the_file.write('publication_short : ""\n')
            
            # Add the abstract if it's available in the bibtex
            if 'abstract' in entry:
                if(len(temp_list)==0):
                    the_file.write('abstract : "'+supetrim(entry['abstract'])+'"\n')
                else:
                    the_file.write('abstract : "'+supetrim(temp_list[0]['summary'])+'"\n')
            # Some features are disabled. I activate them later
            the_file.write('summary : ""\n')
            if 'featured' in entry:
                the_file.write('featured : true\n')
            else:
                the_file.write('featured : false\n')
            the_file.write('projects : []\n')
            the_file.write('slides : ""\n')

            # I add urls to the pdf and the DOI venue = {Eleventh international aaai …},
            #the_file.write('url_pdf : "/publication/'+entry['ID']+'/manuscript.pdf"\n')
            the_file.write('url_pdf : ""\n')
            if 'doi' in entry:
                the_file.write('doi : "'+supetrim(entry['doi'])+'"\n')
            
            
            
            if(len(temp_list)!=0):
                
                try:
                    url=re.search("(?P<url>https?://[^\s]+)", temp_list[0]['arxiv_comment']).group("url")
                    print("the code at,",url)
                    the_file.write('url_code : "'+supetrim(url)+'"\n')
                
                except:
                    the_file.write('url_code : ""\n')
            
            
            
            the_file.write('url_dataset : ""\nurl_poster : ""\n')
            
            
            try:
                the_file.write('url_source : '+supetrim(entry['eprint'])+'\n')
            except KeyError:
                the_file.write('url_source : ""\n')
                
            the_file.write('url_slides : ""\nurl_video : ""\n')
            
            
            # Default parameters that can be later customized
            #the_file.write('math : true\n')
            #the_file.write('highlight : true\n')
            the_file.write('image :\n')
            the_file.write(' caption : ""\n')
            the_file.write(' focal_point: ""\n')
            the_file.write(' preview_only: false\n')

            
            # I keep in my bibtex file a parameter called award for publications that received an award (e.g., best paper, etc.)
            if 'award' in entry:
                the_file.write('award : "true"\n')
            
            # I put the individual .bib entry to a file with the same name as the .md to create the CITE option
            db = BibDatabase()
            db.entries =[entry]
            writer = BibTexWriter()
            with open('content/publication/'+entry['ID']+'/cite.bib', 'w', encoding='utf8') as bibfile:
                bibfile.write(writer.write(db))

            the_file.write('---\n\n')
            the_file.write('### Main contributions\n')
            the_file.write('Coming soon \n')
            the_file.write('### Limitations\n')
            the_file.write('Coming soon \n')
            the_file.write('### Future Directions\n')
            the_file.write('Coming soon \n')
            
            
            # Any notes are copied to the main document
            

In [14]:
import glob
all_bib_files=glob.glob('bib_folder/own_papers*.bib')

In [15]:
all_bib_files

['bib_folder/own_papers_2016_2021.bib']

In [19]:
for file in all_bib_files:
    key=[file.split("/")[1].split('_')[0]]
    if(key==['hate speech'] or key ==['bias hate speech'] or key==['hate crime']):
        key=None
    if(key==['own']):
        key=['Our papers']
    
    
    print(key)
    bibtomark(file,tags=key)

['Our papers']
bib_folder/own_papers_2016_2021.bib
Parsing saha2021short
arxiv: "Short is the Road that Leads from Fear to Hate": Fear Speech in Indian
  WhatsApp Groups
gscholar: Short is the Road that Leads from Fear to Hate: Fear Speech in Indian WhatsApp Groups
++++++++++++++


In [ ]:
with open("author.json", "w") as outfile: 
    outfile.write(json.dumps(author_dict,indent=4))

In [ ]:
#

In [ ]:
all_bib_files[0].split("/")[1].split('_')

In [ ]:
all_bib_files